In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import matplotlib.cm as mplcm
import matplotlib.colors as colors
import bct
import community
from pathlib import Path
import glob
import random

from sklearn.metrics import adjusted_rand_score

%matplotlib inline

# Styles
sns.set_context('poster')
sns.set_style('darkgrid')

plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower

In [ ]:
def subjectmatrix2graphslist(filename, densities):
    """
    Toma un archivo csv con la matriz de input y devuelve una lista de grafos con la densidad de aristas correspondiente
    """

    correlation_matrix = pd.read_csv(
        filename,
        header=None
    )
    n = correlation_matrix.shape[0]
    correlation_matrix -= np.diag(np.ones(n))
    max_links = (n*n-n)//2
    tril_idx = np.tril_indices(n,-1)
    sorted_correlations = sorted(
        correlation_matrix.values[tril_idx].flatten(),
        reverse=True
    )

    Gs = []
    for d in densities:
        idx = int(d*max_links)
        threshold = sorted_correlations[idx]
        Gs.append(nx.from_pandas_adjacency(correlation_matrix>=threshold))

    return Gs

In [ ]:
def extraer_lista_grafos(tipo_red, densities):

    path = r'../resources/DataSujetos' # use your path
    all_files = glob.glob(path + "/{}_suj*.csv".format(tipo_red))


    graphs = {}
    for filename in all_files:
        Gs = subjectmatrix2graphslist(filename, densities)
        name = Path(filename).stem
        graphs[name] = Gs

    return graphs

In [ ]:
densities = np.linspace(0.0,0.4,10)

G_N1 = extraer_lista_grafos("N1", densities)
G_N2 = extraer_lista_grafos("N2", densities)
G_N3 = extraer_lista_grafos("N3", densities)
G_W = extraer_lista_grafos("W", densities)

In [ ]:
pos_N1 = nx.spring_layout(G_N1,k=0.2)
pos_N2 = nx.spring_layout(G_N2,k=0.2)
pos_N3 = nx.spring_layout(G_N3,k=0.2)
pos_W = nx.spring_layout(G_W,k=0.2)

In [ ]:
modulos = community.community_louvain.best_partition(G_N1)
values_N1 = [v for n,v in modulos.items()]
nx.draw(
    G_N1,
    pos=pos_N1,
    with_labels=True,
    cmap=plt.cm.tab10,
    font_weight='bold',
    node_color = values_N1,
    font_color='white',
    edge_color='gray'
)

In [ ]:
def partition_set_to_dict(m):
    d = {}
    for i,c in enumerate(m):
        for n in c:
            d[n] = i
    return d

In [ ]:
modulos = nx.community.girvan_newman(G_N1)
modulos = list(modulos)
modularity = []
for m in modulos:
    modularity.append(
        community.modularity(
            partition_set_to_dict(m),
            G_N1
        )
    )
plt.plot(modularity)
plt.xlabel('Diferentes particiones')
plt.ylabel('Coeficiente de Modularidad')
idx = np.argmax(modularity)
m = modulos[idx]

In [ ]:
plt.figure()
values = [v for n,v in partition_set_to_dict(m).items()]
nx.draw(
    G_N1,
    pos=pos,
    with_labels=True,
    cmap=plt.cm.tab10,
    font_weight='bold',
    node_color = values,
    font_color='white',
    edge_color='gray'
)

In [ ]:
itero = 20
R_mod=[]
R_num=[]

for i in range(itero):
    G_R = nx.gnm_random_graph(G_W.number_of_nodes(),G_W.number_of_edges())
    modulos_R_L = community.best_partition(G_R)
    R_mod.append(community.modularity(modulos_R_L,G_R))
    R_num.append(max(modulos_R_L.values())+1)

print("ended")

In [ ]:
  def randomize_by_edge_swaps(graph , num_iterations):

        newgraph = graph.copy()
        edge_list = list(newgraph.edges())

        num_edges = len(edge_list)
        total_iterations = num_edges * num_iterations

        for i in range(total_iterations):
            rand_index1 = int(round(random.random() * (num_edges - 1)))
            rand_index2 = int(round(random.random() * (num_edges - 1)))
            original_edge1 = edge_list[rand_index1]
            original_edge2 = edge_list[rand_index2]
            head1, tail1 = original_edge1
            head2, tail2 = original_edge2

            if random.random() >= 0.5:
                head1, tail1 = tail1, head1


            if head1 == tail2 or head2 == tail1:
                continue

            if newgraph.has_edge(head1, tail2) or newgraph.has_edge(
                    head2, tail1):
                continue

            # Suceeded checks, perform the swap
            original_edge1_data = newgraph[head1][tail1]
            original_edge2_data = newgraph[head2][tail2]

            newgraph.remove_edges_from((original_edge1, original_edge2))

            new_edge1 = (head1, tail2, original_edge1_data)
            new_edge2 = (head2, tail1, original_edge2_data)
            newgraph.add_edges_from((new_edge1, new_edge2))

            # Now update the entries at the indices randomly selected
            edge_list[rand_index1] = (head1, tail2)
            edge_list[rand_index2] = (head2, tail1)

        assert len(newgraph.edges()) == num_edges
        return newgraph